In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
proj_path = '/content/drive/MyDrive/Model Development '
os.chdir(proj_path)
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1_QHmZZAWiu1Idc_qP56o_8AWHyZMtSIR/Model Development '

In [ ]:
#Imports
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Concatenate
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
!pip install -q pydot
!pip install graphviz
!pip install pydotplus
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#image sizing
IMAGE_SIZE=[224,224,3]
train_path='/content/drive/MyDrive/Model Development /tbdataset/Train'
valid_path='/content/drive/MyDrive/Model Development /tbdataset/Test'

In [ ]:
# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Preprocessing the Train set
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Model Development /tbdataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Model Development /tbdataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 3167 images belonging to 2 classes.
Found 791 images belonging to 2 classes.


In [ ]:
IMAGE_SIZE = (224,224,3)

vgg19 = tf.keras.applications.vgg19.VGG19(
    input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
for layer in vgg19.layers:
    layer._name = layer._name + str('_19')
    layer.trainable = False

vgg16 = tf.keras.applications.vgg16.VGG16(
    input_shape=IMAGE_SIZE, weights='imagenet', include_top=False)
for layer in vgg16.layers:
    layer._name = layer._name + str('_16')
    layer.trainable = False

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
inp = Input(IMAGE_SIZE)
    
vgg16_x = Flatten()(vgg16(inp))
vgg19_x = Flatten()(vgg19(inp))

The following Variables were used a Lambda layer's call (tf.nn.convolution), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.bias_add), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_1), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv2/kernel:0' shape=(3, 3, 64, 64) dtype=float32>
It is possible that this is int

In [ ]:
x = Concatenate()([vgg16_x, vgg19_x])
out = Dense(2, activation='softmax')(x)

model = Model(inputs = inp, outputs = out)
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    name="Adam"),
  metrics=['accuracy',
           'AUC',
           'Precision',
           'Recall',
  ]
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf.nn.convolution_13 (TFOpLambd (None, 224, 224, 64) 0           input_1[0][0]                    
__________________________________________________________________________________________________
tf.nn.bias_add_13 (TFOpLambda)  (None, 224, 224, 64) 0           tf.nn.convolution_13[0][0]       
__________________________________________________________________________________________________
tf.nn.relu_13 (TFOpLambda)      (None, 224, 224, 64) 0           tf.nn.bias_add_13[0][0]          
______________________________________________________________________________________________

In [ ]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/20
99/99 [==============================] - 2590s 26s/step - loss: 0.5691 - accuracy: 0.7390 - auc: 0.8041 - precision: 0.7390 - recall: 0.7390 - val_loss: 0.2393 - val_accuracy: 0.9140 - val_auc: 0.9651 - val_precision: 0.9140 - val_recall: 0.9140
Epoch 2/20
99/99 [==============================] - 217s 2s/step - loss: 0.2707 - accuracy: 0.8974 - auc: 0.9545 - precision: 0.8974 - recall: 0.8974 - val_loss: 0.2089 - val_accuracy: 0.9267 - val_auc: 0.9728 - val_precision: 0.9267 - val_recall: 0.9267
Epoch 3/20
99/99 [==============================] - 206s 2s/step - loss: 0.2311 - accuracy: 0.9081 - auc: 0.9675 - precision: 0.9081 - recall: 0.9081 - val_loss: 0.2062 - val_accuracy: 0.9178 - val_auc: 0.9742 - val_precision: 0.9178 - val_recall: 0.9178
Epoch 4/20
99/99 [==============================] - 213s 2s/step - loss: 0.2148 - accuracy: 0.9041 - auc: 0.9718 - precision: 0.9041 - recall: 0.9041 - val_loss: 0.3614 - val_accuracy: 0.8559 - val_auc: 0.9370 - val_precision: 0.8559

In [ ]:
model.save('saved_models/ensemblevgg') 

AssertionError: ignored

In [ ]:
predictions = model.predict(test_set, steps = test_set.n // 31, verbose=1)

25/25 [==============================] - 26s 1s/step


In [ ]:
y_classes = predictions.argmax(axis=1)

In [ ]:
test_set.reset()

In [ ]:
y_classes

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print('Confusion Matrix:')
print(confusion_matrix(test_set.classes, y_classes))

print("Accuracy : " ,accuracy_score(test_set.classes, y_classes))
print("Precision : " ,precision_score(test_set.classes, y_classes))
print("Recall : " ,recall_score(test_set.classes, y_classes))
print("F1 Score : " ,f1_score(test_set.classes, y_classes))

Confusion Matrix:
[[389   8]
 [ 47 347]]
Accuracy :  0.9304677623261695
Precision :  0.9774647887323944
Recall :  0.8807106598984772
F1 Score :  0.9265687583444593


In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, threshold = roc_curve(test_set.classes, y_classes)
print("SVM Area under curve -> ",auc(fpr, tpr))

SVM Area under curve ->  0.9302797631986089
